# Plan dnia

1. Problem nierównolicznych klas

2. Nowe klasyfikatory

3. Case study


<br>

<br>

<br>

<br>

<br>

<br>

<br>

# 1. Krzywa ROC

Mamy wektor prawdziwych klas $y = [0, 1, 1, ..., 0, 0]$ oraz wektor prawdopodobieństw zwrócony przez klasyfikator $p = [0.35, 0.55, 0.91, ..., 0.1, 0.44]$. Klasyfikujemy obserwacje według reguły $p(x) > t$ dla pewnego progu $t$.

Zdefiniujmy:
 - _false positive rate_ FPR - procent obserwacji klasy $0$ zaklasyfikowanych jako $1$,
 - _true positive rate_ TPR - procent obserwacji klasy $1$ zaklasyfikowanych jako $1$.

Obliczamy TPR($t$) oraz FPR($t$) dla każdego $t\in p$. Krywa ROC to wykres liniowy powstały z punktów (FPR($t$), TPR($t$)) (dla $t$ uporządkowanych malejąco).

Wskaźnik AUC to wielkość pola pod krzywą ROC. Wskaźnik AUC jest dobrą miara dla niezbalansowanych klas (podobnie jak F1).

<img src="http://scikit-learn.org/stable/_images/sphx_glr_plot_roc_001.png" width="400">

Źródło: http://scikit-learn.org/stable/_images/sphx_glr_plot_roc_001.png

In [ ]:
import numpy as np
from sklearn.metrics import roc_curve, roc_auc_score
y = np.array([0, 0, 1, 1, 1])
scores = np.array([0.01, 0.4, 0.35, 0.8, 0.6])
fpr, tpr, thresholds = roc_curve(y, scores)
print(thresholds)
print(fpr)
print(tpr)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.plot(fpr,tpr, label = "AUC = %.2f" % roc_auc_score(y,scores))
plt.xlabel("FDR")
plt.ylabel("TPR")
plt.title("ROC")
plt.plot([0, 1], [0, 1], linestyle='--')
plt.legend(loc="lower right", prop={'size':15})
plt.show()

# 2. Problem nierównomierności klas (_imbalanced data_)

https://svds.com/learning-imbalanced-classes/

Podejścia:

- "ważona" funkcja celu
- over-sampling
- under-sampling
- data-augmentation (przede wszystkim przy pracy z obrazami)
- podejścia inne niż klasyfikacja statystyczna (detekcja obserwacji nietypowych, własne heurystyki)

Zawsze można też manipulować progiem klasyfikacji.

In [ ]:
import pandas as pd

Opis danych:

It contains only numerical input variables which are the result of a PCA transformation. Unfortunately, due to confidentiality issues, we cannot provide the original features and more background information about the data. Features V1, V2, ... V28 are the principal components obtained with PCA, the only features which have not been transformed with PCA are 'Time' and 'Amount'. Feature 'Time' contains the seconds elapsed between each transaction and the first transaction in the dataset. The feature 'Amount' is the transaction Amount.

In [ ]:
data = pd.read_csv("Dane/creditcard.csv")
data.head()

In [ ]:
np.unique(data.Class,return_counts=True)

In [ ]:
np.mean(data.Class)

### Przygotowanie zbioru :

1. Co zrobić ze zmienną Time?

2. Przeanalizuj rozkład zmiennej Amount (w tym celu narysuj histogram oraz boxplot). Zaproponuj jest transformację.

In [ ]:
data.drop(['Time'],axis=1,inplace=True)

In [ ]:
plt.hist(data.Amount)
plt.show()

In [ ]:
plt.hist(data.Amount,bins=[0,200,500,1000,5000,25000])
plt.show()

In [ ]:
plt.boxplot(data.Amount)
plt.show()

In [ ]:
np.min(data.Amount)

In [ ]:
data['Amount'] = np.log(1+data["Amount"])

plt.hist(data.Amount)
plt.show()

In [ ]:
data.describe()

In [ ]:
X = data.drop(["Class"],axis=1,inplace=False)
Y = data.Class

In [ ]:
from sklearn.model_selection import cross_val_predict, StratifiedKFold

Przykład: Regresja logistyczna

Zawsze zaczynamy od określenia punktu odniesienia!

In [ ]:
from sklearn.linear_model import LogisticRegression
c = LogisticRegression()
preds = cross_val_predict(estimator=c, X=X, y=Y, method="predict_proba",cv = StratifiedKFold(3))[:,1]
roc_auc_score(Y,preds)

Ważenie:

In [ ]:
c = LogisticRegression(class_weight={0:1,1:100})
preds = cross_val_predict(estimator=c, X=X, y=Y, method="predict_proba",cv = StratifiedKFold(5))[:,1]
roc_auc_score(Y,preds)

In [ ]:
c = LogisticRegression(class_weight={0:1,1:577})
preds = cross_val_predict(estimator=c, X=X, y=Y, method="predict_proba",cv = StratifiedKFold(5))[:,1]
roc_auc_score(Y,preds)

Zadanie powtórz analizę dla drzewa decyzyjnego

In [ ]:
from sklearn.tree import DecisionTreeClassifier
c = DecisionTreeClassifier()
preds = cross_val_predict(estimator=c, X=X, y=Y, method="predict_proba",cv = StratifiedKFold(3))[:,1]
roc_auc_score(Y,preds)

In [ ]:
c = DecisionTreeClassifier(class_weight={0:1,1:100})
preds = cross_val_predict(estimator=c, X=X, y=Y, method="predict_proba",cv = StratifiedKFold(5))[:,1]
roc_auc_score(Y,preds)

### Zadanie: przetestuj undersamplig.

In [ ]:
import numpy as np

In [ ]:
np.mean(Y[:100000])

In [ ]:
from copy import deepcopy

def cross_validate_undersampling(X, Y, model, size=1, cv=StratifiedKFold(3,random_state=1)):
    
    """
    X - ramka danych
    Y - pd.Series
    size - określa stosunek liczby obserwacji klasy 0 do klasy 1 w zbiorze wynikowym
    """
    
    
    preds = []
    true_labels = []
    
    for train_index, test_index in cv.split(X,Y):
        
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = Y.iloc[train_index], Y.iloc[test_index]
    

        ind0 = np.random.choice(np.where(y_train==0)[0], int(size*np.sum(y_train==1)),replace=False)  # (size * liczba oserwacji klasy 1) losowych indeksów obserwacji klasy 0 (bez powtórzeń)
        ind_final = np.r_[ind0, np.where(y_train==1)[0]] # indeksy ind0 i indeksy klasy obserwacji (można wykorzystać np.r_[...])
        X_train_subsample = X_train.iloc[ind_final]
        y_train_subsample = y_train.iloc[ind_final]
        
        clf = deepcopy(model)
        clf.fit(X_train_subsample, y_train_subsample) # uczymy model stworzonym zbiorze
               
        preds.extend(clf.predict_proba(X_test)[:,1]) # prawdopodobienstwa na zbiorze testowym
        true_labels.extend(y_test)

    return roc_auc_score(true_labels, preds)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score

for model in [LogisticRegression(),DecisionTreeClassifier()]:
    for size in [1,2,5,0.5]:
        print(cross_validate_undersampling(X,Y,model,size))

In [ ]:
from copy import deepcopy

def cross_validate_undersampling(X, Y, model, size=1, cv=StratifiedKFold(5,random_state=1)):
    
    preds = []
    true_labels = []
    
    for train_index, test_index in cv.split(X,Y):
        
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = Y.iloc[train_index], Y.iloc[test_index]
    

        ind0 = np.random.choice(np.where(y_train==0)[0],int(size*sum(y_train==1)),replace=False)
        ind_final = np.r_[ind0, np.where(y_train==1)[0]]
        X_train_subsample = X_train.iloc[ind_final]
        y_train_subsample = y_train.iloc[ind_final]
        
        clf = deepcopy(model)
        clf.fit(X_train_subsample,y_train_subsample)
               
        preds.extend(clf.predict_proba(X_test)[:,1])
        true_labels.extend(y_test)

    return roc_auc_score(true_labels,preds)

In [ ]:
for model in [LogisticRegression(),DecisionTreeClassifier()]:
    for size in [1,2,5,10,0.2]:
        print(cross_validate_undersampling(X,Y,model,size))

### Zadanie: przetestuj oversampling. Napisz w tym celu funkcję analogiczną do funkcji testującej undersampling. Uwaga: obliczenia potrwają dłużej!

In [ ]:
from copy import deepcopy

def cross_validate_oversampling(X, Y, model, size=1, cv=StratifiedKFold(5,random_state=1)):
    
    preds = []
    true_labels = []
    
    for train_index, test_index in cv.split(X,Y):
        
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = Y.iloc[train_index], Y.iloc[test_index]
    

        ind1 = np.resize(np.where(y_train==1)[0], int(size*np.sum(y_train==1)))
        ind_final = np.r_[ind1, np.where(y_train==0)[0]]
        X_train_subsample = X_train.iloc[ind_final]
        y_train_subsample = y_train.iloc[ind_final]
        
        clf = deepcopy(model)
        clf.fit(X_train_subsample,y_train_subsample)
               
        preds.extend(clf.predict_proba(X_test)[:,1])
        true_labels.extend(y_test)

    return roc_auc_score(true_labels,preds)

In [ ]:
for model in [LogisticRegression(),DecisionTreeClassifier()]:
    for size in [1,0.5]:
        print(cross_validate_undersampling(X[:50000],Y[:50000],model,size))

### Zadanie: przetestuj modyfikowany undersampling: k-krotnie dokonujemy predykcji przy użyciu undersamplingu, a następnie ostateczna predykcja to średnia z prawdopodobieństw k predykcji.

In [ ]:
def cross_validate_repeated_undersampling(X, Y, model, n_estimators=3, cv=StratifiedKFold(5,random_state=1)):
    
    preds = []
    true_labels = []
    size=1
        
    for train_index, test_index in cv.split(X,Y):
        
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = Y.iloc[train_index], Y.iloc[test_index]
    
        scores = np.zeros((n_estimators,len(y_test)))
        for i in range(n_estimators):
            ind0 = np.random.choice(np.where(y_train==0)[0],int(size*sum(y_train==1)),replace=False)
            ind_final = np.r_[ind0, np.where(y_train==1)[0]]
            X_train_subsample = X_train.iloc[ind_final]
            y_train_subsample = y_train.iloc[ind_final]

            clf = deepcopy(model)
            clf.fit(X_train_subsample,y_train_subsample)
            
            scores[i,:] = clf.predict_proba(X_test)[:,1]
            #print(scores)

        preds.extend(scores.mean(0))
        #print(scores)
        #print(preds)
        true_labels.extend(y_test)

    return roc_auc_score(true_labels,preds)

In [ ]:
for model in [LogisticRegression(),DecisionTreeClassifier()]:
    for k in [3, 10]:
        print(cross_validate_repeated_undersampling(X,Y,model,n_estimators=k))

# Klasyfikatory: 

## LDA (_linear discriminant analysis_)   &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;          QDA (_quadratic discriminant analysis_)


Korzystamy ze wzoru Bayesa:

## $$P(Y=k|x) \sim P(x|Y=k)P(Y=k)$$

### $P(x|Y=k)$ - wyestymowany rozkład wielowymiarowy normalny. (LDA zakłada jednakową macierz kowariancji w podgrupach, QDA - różne macierze)
### $P(Y=k)$ - prawdopodobieństwo a priori.

### Klasyfikujemy do klasy, która ma największe prawdopodobieństwo $P(Y=k|x)$.



<img src="https://nauka.metodolog.pl/wp-content/uploads/2016/05/liniowa-i-kwadratowa-funkcja-dyskryminacyjna.png" width="500">
Źródło: http://nauka.metodolog.pl/liniowa-analiza-dyskryminacji/

<img src="https://www.researchgate.net/profile/Alexander_Fotheringham/publication/229909322/figure/fig9/AS:295501053349908@1447464349480/Figure-1-Discriminant-analysis-illustrated-LHS-shows-the-result-of-LDA-applied-to-the.png" width="700">

# Podejścia bezmodelowe do klasyfikacji

## Metoda K Najbliższych sąsiadów (KNN)

<img src="https://i.pinimg.com/originals/65/36/b9/6536b9a63fc427e0fc3e1a9687b49aff.png" width="500">


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
?KNeighborsClassifier

ciekawy argument - _weights_

### Metoda najbliższego centroidu

In [ ]:
from sklearn.neighbors import NearestCentroid
?NearestCentroid

W praktyce bardzo rzadko stosowana (o ile w ogóle:))

Pytanie: dla jakich danych metoda ma sens?

- kiedy jesteśmy w stanie sensownie określić metrykę i centroid ma sens

# Zadanie - dyskusja w parach:

Załóżmy, że w naszych danych obserwacje klasy 1 stanowią 1% zbioru, a naszym zadaniem jest klasyfikacja. Zaproponuj podejście do problemu oparte na analizie najbliższych sąsiadów.

# Zadanie

Bierzemy dane:
https://archive.ics.uci.edu/ml/datasets/Bank+Marketing#

bank-full

1) Przetwórz dane do postaci, którą przyjmą modele (zamień zmienne nominalne na mumeryczne - one-hot-encoding)

2) Podziel dane na zbiór uczący, walidacyjny i testowy z zachowaniem proporcji klas (google -> stratified train test split sklearn (można użyć dwukrotnie podziału na uczacy/testowy))

3) Na zbiorze uczącym przy użyciu krowalidacji znajdź optymalne (miara f1) parametry modeli (gridsearchCV):

 - regresja logistyczna: type regularyzacji, współczynnik regularyzacji
 - drzewo decyzyjne: minimalna liczba obserwacji w liściu,
 - LDA (tylko parametey domyślne - przekazujemy grid search'owi pusty słownik)
 - QDA (tylko parametey domyślne - przekazujemy grid search'owi pusty słownik)
 - lasy losowe: liczba drzew
 - K najbliższych sąsiadów: liczba sąsiadów

 W tym celu:
 - stwórz listę modeli (listę wywołań konstruktorów z domyślnymi parametrami (postaci: [ KMeans(), ...] ))
 - stwórz listę siatek parametrów - jeden element listy to siatka parametrów (w postaci jakiej wymaga gridsearchCV) dla odpowiadającego modelu
 - przejdź pętlą po parach model,siatka (przyda się funkcja zip), wypisz w każdej iteracji wyniki (grid_scores_) kroswalidacji (przyda się funkcja pprint z modułu pprint) i zapisz w liście najlepsze parametry (.best\_params\_) dla każdego modelu

 Możesz poszerzyć siatki o inne parametry.


4) Przetestuj każdy model z najlepszymi parametrami na zbiorze validacyjnym, przechodząc pętlą po parach model,zestaw_najlepszych_parametrow (wykorzystaj operację \**_dict_ do nadania parametrów na podstawie słownika _dict_) i wypisz Accuracy, miarę F1 oraz AUC otrzymane na zbiorze walidacyjnym

5) Przetestuj na zbiorze validacyjnym "model warstwowy" względem zmiennej housing, konstruowany w następujący sposób: uczymy model (np. drzewo decyzyjne) w podgrupach wyznaczonych przez zmienną housing - tzn. uczymy jedno drzewo na podzbiorze, w którym housing=Yes, a drugie w podzbiorze housing=No. Następnie dokonujemy predykcji w taki sposób, że dla nowej obserwacji patrzymy na wartość zmiennej housing i w zależności od jej wartości dokonujemy predykcji drzewem nauczonym na odpowiednim podzbiorze.

6) Wykorzystanie analizy skupień w klasyfikacji. Zbadaj dwa podejścia:
  - Pogrupuj dane algorytmem k-średnich. Dodaj do danych zmienną NOMINALNĄ - kodowaną one-hot (nie numeryczną), której wartości są etykietami grupy obserwacji. Na danych rozszerzonych o tę zmienną powtórz punkty 3 i 4 (nie zmieniaj wcześniej przyjętych siatek parametrów). Przetestuj w kroswalidacji różne liczby skupień, a następnie przestestuja na zbiorze walidacyjnym kilka najlepszych modeli. UWAGA: pogrupowanie musi się odbyć na zbirze treningowym, natomiast wartości tej zmiennej na zbiorze walidacyjnym otrzymujemy przyporządowując obserwacje do najbliższych klastrów! [Zadanie z \*: wykonaj analogiczną procedurę dla grupowania hierarchicznego - w tym przypadku nie dysponujemy metodą predict - samemu trzeba zaimplementować przyporządkowywanie obserwacji do klastrów).
  - Pogrupuj dane algorytmem k-średnich. W każdym klastrze z osobna dopasuj model klasyfikacji (ten sam model uczony oddzielnie na każdym podzbiorze). Dokonuj predykcji obserwacji w następujący sposób: przypisz obserwację do najbliższego klastra, a następnie dokonaj predykcji przy użyciu modelu uczonego w tym klastrze.

7) Przetestuj over/under sampling

8) Przetestuj na zbiorze validacyjnym transformację danych przy użyciu PCA - powtórz stworzony do tej pory kod zmieniając go tak, aby na starcie zamiast inicjacji modeli były pipeliny PCA+Model. 
   

# 9) Oceń jakość działania pięciu najlepszych modeli/pipeline'ów na zbiorze testowym

10*) Spróbuj przetworzyć oryginalne zmienne do lepszej postaci. Powtórz cały proces w celu przetestowania wpływu sposobu przetworzenia danych. 


In [ ]:
import pandas as pd
data = pd.read_csv('Dane/bank/bank-full.csv',sep=';')
data.head()

In [ ]:
data.describe(include="all")

In [ ]:
data.drop(["day","month","duration"],axis=1,inplace=True)
data = pd.get_dummies(data,drop_first=True)

In [ ]:
data.head()

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit
import numpy as np
from sklearn.preprocessing import StandardScaler

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
def split_table(table, labels=y,split = (0.78,0.11,0.11),

                seed = 42):

    '''Splits the table into train, validation and test sets.

       Returns a tuple (X_train, X_val, X_test, y_train, y_val, yest)'''



    # split into training set and the rest

    X_train, X_temp, y_train, y_temp = train_test_split(

        table, labels,

        test_size = 1 - split[0],

        random_state = seed, stratify = labels)



    # split the remaining data into validation and test sets

    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp,

        test_size = split[2]/(split[1]+split[2]), random_state = seed,

        stratify = y_temp)



    return X_train, X_val, X_test, y_train, y_val, y_test

In [ ]:
X_train, X_val, X_test, y_train, y_val, y_test = split_table(data.drop(["y_yes"],axis=1,inplace=False),data.y_yes)

In [ ]:
# mniej sprytny sposób:

X = data.drop(["y_yes"],axis=1,inplace=False)
colnames = X.columns
y = data.y_yes

sss = StratifiedShuffleSplit(2,5000)
ind = list(sss.split(X,y))[0]
print(len(ind))


X_train_tmp, y_train_tmp = X.iloc[ind[0]], y.iloc[ind[0]]
X_test, y_test = X.iloc[ind[1]], y.iloc[ind[1]]

sss = StratifiedShuffleSplit(2,5000)
ind = list(sss.split(X_train_tmp,y_train_tmp))[0]

X_train, y_train = X_train_tmp.iloc[ind[0]], y_train_tmp.iloc[ind[0]]
X_valid, y_valid = X_train_tmp.iloc[ind[1]], y_train_tmp.iloc[ind[1]]

del X_train_tmp, y_train_tmp

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_valid = sc.transform(X_valid)
X_test = sc.transform(X_test)

print(X_train.shape, X_valid.shape, X_test.shape)
print(np.mean(y_train),np.mean(y_valid),np.mean(y_test))

In [ ]:
from sklearn.linear_model import LogisticRegression
#from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
#

models = [LogisticRegression(),
          KNeighborsClassifier(), 
          LinearDiscriminantAnalysis(), 
          QuadraticDiscriminantAnalysis(),
          DecisionTreeClassifier(), 
          RandomForestClassifier()]
param_grids = [{"penalty":["l1","l2"],"C":[0.1,1,10,100,1000,10000]},
              {"n_neighbors":[1,3,7,19]},
              {},
              {},
              {"min_samples_leaf":[5,10,25,50,100]},
              {"n_estimators":[10,100,300,500]}]

In [ ]:
from sklearn.model_selection import GridSearchCV

from pprint import pprint

In [ ]:
models = [LogisticRegression(),
          KNeighborsClassifier(), 
          LinearDiscriminantAnalysis(), 
          QuadraticDiscriminantAnalysis(),
          DecisionTreeClassifier(), 
          RandomForestClassifier()]
param_grids = [{"penalty":["l1","l2"],"C":[0.1,1,10,100,1000,10000]},
              {"n_neighbors":[1,3,7,19]},
              {},
              {},
              {"min_samples_leaf":[5,10,25,50,100]},
              {"n_estimators":[10,100,300,500]}]

best_params = []
for model, grid in zip(models,param_grids):
    
    print(model)
    cv = GridSearchCV(model,grid,scoring="f1")
    cv.fit(X_train,y_train)
    best_params.append(cv.best_params_)
    pprint(cv.grid_scores_)
    
    

for mod, params in zip(models,best_params):

    model = deepcopy(mod)
    model.set_params(**params)
    model.fit(X_train,y_train)
    preds = model.predict_proba(X_val)[:,1]
    print(accuracy_score(y_val,preds>0.5), f1_score(y_val,preds>0.5), roc_auc_score(y_val,preds))

In [ ]:
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score
from copy import deepcopy

for mod, params in zip(models,best_params):

    model = deepcopy(mod)
    model.set_params(**params)
    model.fit(X_train,y_train)
    preds = model.predict_proba(X_val)[:,1]
    print(accuracy_score(y_val,preds>0.5), f1_score(y_val,preds>0.5), roc_auc_score(y_val,preds))

In [ ]:
X_valid = X_val
y_valid = y_val

In [ ]:
ind2

In [ ]:
#TRZEBA PAMIETAC ZE ZMIENNE ZERO JEDYNKOWE SA ZESTANDARYZOWANE I NIE PRZYJMUJA 1 (TRZEBA POROWNYWAC Z ZEREM)

ind1 = (X_train["housing_yes"]>0).reshape(-1)
x_1 = X_train.iloc[ind1].drop("housing_yes",axis=1,inplace=False)
y_1 = y_train.iloc[ind1]
print(np.mean(y_1))

ind2 = (X_train["housing_yes"]<=0).reshape(-1)
x_2 = X_train.iloc[ind2].drop("housing_yes",axis=1,inplace=False)
y_2 = y_train.iloc[ind2]
print(np.mean(y_2))

for model,params in zip(models,best_params):    

    model1 = deepcopy(model)
    model1.set_params(**params)
    model1.fit(x_1,y_1)
  
    
    model2 = deepcopy(model)
    model2.set_params(**params)
    model2.fit(x_2,y_2)
    
    #print(pd.crosstab(y_1,model1.predict(x_1)))
    #print(pd.crosstab(y_2,model1.predict(x_2)))
    
    preds = np.zeros_like(y_valid)
    
    ind1 = (X_valid["housing_yes"]>0).reshape(-1)
    
    #print(colnames!="housing_yes")
    #print(model1.predict_proba(X_valid[ind1][:,colnames!="housing_yes"])[:,1])
    preds[ind1] = model1.predict_proba(X_valid.iloc[ind1].drop("housing_yes",axis=1,inplace=False))[:,1]
    
    ind2 = (X_valid["housing_yes"]<=0).reshape(-1)
    preds[ind2] = model2.predict_proba(X_valid.iloc[ind2].drop("housing_yes",axis=1,inplace=False))[:,1]
    
    #print(accuracy_score(y_valid))
    #print(preds[:10])
    print(accuracy_score(y_valid,preds>0.5), f1_score(y_valid,preds>0.5), roc_auc_score(y_valid,preds))
    

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
for k in [2,3]:
    
    print("k = ", k)
    
    km = KMeans(k)
    X_train2 = np.concatenate([X_train, pd.get_dummies(km.fit_predict(X_train),drop_first=True)],axis=1)
    X_valid2 = np.concatenate([X_valid, pd.get_dummies(km.predict(X_valid),drop_first=True)],axis=1)
    X_test2 = np.concatenate([X_test, pd.get_dummies(km.predict(X_test),drop_first=True)],axis=1)
    
    for mod, params in zip(models,best_params):

        model = deepcopy(mod)
        model.set_params(**params)
        model.fit(X_train2,y_train)
        preds = model.predict_proba(X_valid2)[:,1]
        print(accuracy_score(y_valid,preds>0.5), f1_score(y_valid,preds>0.5), roc_auc_score(y_valid,preds))

In [ ]:
for K in [2,3]:
    
    print("K = ", K)
    
    km = KMeans(K)
    train_clusters = km.fit_predict(X_train)
    valid_clusters = km.predict(X_valid)

    for model,params in zip(models,best_params):

        preds = np.zeros_like(y_valid)

        for k in range(K):

            ind = train_clusters==k
            x = X_train[ind]
            y = y_train[ind]

            mod = deepcopy(model)
            mod.set_params(**params)
            mod.fit(x,y)

            preds[valid_clusters==k] = mod.predict_proba(X_valid[valid_clusters==k])[:,1]
            #print(preds[:10])
        print(accuracy_score(y_valid,preds>0.5), f1_score(y_valid,preds>0.5), roc_auc_score(y_valid,preds))


In [ ]:
from sklearn.pipeline import Pipeline

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
best_models = [model.set_params(**params) for model,params in zip(models,best_params)]
pipelines = [Pipeline([("pca",PCA()),("model",model)]) for model in best_models]

In [ ]:
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score
from copy import deepcopy

for k in [10,20,30,40]:
    print(k)
    for pipe in pipelines:

        model = deepcopy(pipe)
        model.set_params(pca__n_components=k)
        model.fit(X_train,y_train)
        preds = model.predict_proba(X_valid)[:,1]
        print(accuracy_score(y_valid,preds>0.5), f1_score(y_valid,preds>0.5), roc_auc_score(y_valid,preds))